In [93]:
import pandas as pd
import operator
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('Iris.csv', index_col='Id')

### 2.1. Exiba a quantidade de amostras existente em cada classe.

In [4]:
df['Species'].value_counts()

Iris-setosa        50
Iris-virginica     50
Iris-versicolor    50
Name: Species, dtype: int64

### 2.2. Separe aleatoriamente 80% dos dados para treino (conjunto de treino) e 20% dos dados para teste (conjunto de teste). Para esta separação, use somente recursos do Python, Numpy ou Pandas. Não use o Scikit Learn para isso.

In [5]:
X = df.drop(columns='Species', axis=1)
y = df['Species']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

### 2.3. Usando somente o conjunto de treino, mostre as médias dos valores de cada feature para cada classe.

In [23]:
class_info = pd.concat([X_train, y_train], axis=1).groupby('Species').agg({
    'PetalLengthCm': ['mean', 'std',sum, max, min], 
    'PetalWidthCm': ['mean', 'std',sum, max, min],
    'SepalLengthCm': ['mean', 'std',sum, max, min],
    'SepalWidthCm': ['mean', 'std',sum, max, min],
})

In [24]:
class_info

PetalLengthCm                            PetalWidthCm  \
                         mean       std    sum  max  min         mean   
Species                                                                 
Iris-setosa          1.453659  0.165978   59.6  1.9  1.0      0.24878   
Iris-versicolor      4.233333  0.443364  152.4  5.1  3.3      1.30000   
Iris-virginica       5.548837  0.528868  238.6  6.9  4.5      2.02093   

                                          SepalLengthCm                        \
                      std   sum  max  min          mean       std    sum  max   
Species                                                                         
Iris-setosa      0.109822  10.2  0.6  0.1      5.014634  0.360944  205.6  5.8   
Iris-versicolor  0.197122  46.8  1.8  1.0      5.919444  0.533891  213.1  7.0   
Iris-virginica   0.285822  86.9  2.5  1.4      6.546512  0.604487  281.5  7.9   

                     SepalWidthCm                             
                 min         mean       std    sum  max  min  
Species                                                       
Iris-setosa      4.4     3.402439  0.381109  139.5  4.4  2.3  
Iris-versicolor  4.9     2.783333  0.303786  100.2  3.4  2.0  
Iris-virginica   4.9     2.969767  0.317381  127.7  3.8  2.2

### 2.4. Crie um algoritmo seu (sem usar aprendizado de máquina) para baseado nas médias das features calculadas no item 2.3. ou mesmo usando outras ideias suas, classificar corretamente as amostras do conjunto de teste. Aperfeçoe o seu algoritmo de forma a obter melhores resultados. Quanto por cento no conjunto de teste você conseguiu acertar corretamente o rótulo (classe) no seu melhor algoritmo, ou seja, qual a acurácia?

In [140]:
def classificationAlgorithm(X_predict):
    result = {}
    for index, predict in X_predict.iterrows():
        PetalLengthCm, PetalWidthCm, SepalLengthCm, SepalWidthCm = predict
        pesos = {'Iris-setosa': 0,'Iris-versicolor': 0, 'Iris-virginica': 0}
        try: 
            pesos[class_info['PetalLengthCm'].query('not(min >= @PetalLengthCm)').index[-1]] += 1
            pesos[class_info['PetalLengthCm'].query('not(max < @PetalLengthCm)').index[-1]] += 1
        except:
            pesos['Iris-setosa'] += 1

        try: 
            pesos[class_info['PetalWidthCm'].query('not(min >= @PetalWidthCm)').index[-1]] += 1
            pesos[class_info['PetalWidthCm'].query('not(max < @PetalWidthCm)').index[-1]] += 1
        except:
            pesos['Iris-setosa'] += 1

        try: 
            pesos[class_info['SepalLengthCm'].query('not(min >= @SepalLengthCm)').index[-1]] += 1
            pesos[class_info['SepalLengthCm'].query('not(max < @SepalLengthCm)').index[-1]] += 1
        except:
            pesos['Iris-setosa'] += 1

        try: 
            pesos[class_info['SepalWidthCm'].query('not(min >= @SepalWidthCm)').index[-1]] += 1
            pesos[class_info['SepalWidthCm'].query('not(max < @SepalWidthCm)').index[-1]] += 1
        except:
            pesos['Iris-setosa'] += 1
        
        result[index] = max(pesos.items(), key=operator.itemgetter(1))[0]
        
    return pd.DataFrame(result.values(), index = list(result.keys()), columns=['predict'])

In [141]:
y_predict = classificationAlgorithm(X_test)

In [142]:
metrics.accuracy_score(y_predict, y_test)

0.5333333333333333